In [1]:
'''
aim to deal with long term business closure
idea:
step1: get opened business id in round 7, and locate them by business id to round 10 business dataset
step2: get corresponding business id(balanced open-open, open-closed), and fetch corrsponding review from round7 review
'''
import pandas as pd
import numpy as np
df_busi_round7 = pd.read_table('/Users/luxiaopeng/notebook/inls613/closure_prediction/round7/business_round7.txt')
df_busi_round10 = pd.read_table('/Users/luxiaopeng/notebook/inls613/closure_prediction/round10/business_round10.txt')
df_review_round7 = pd.read_table('/Users/luxiaopeng/notebook/inls613/closure_prediction/round7/review_round7.txt')

In [138]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [27]:
# first get those who are open
df_busi_round7_open = df_busi_round7[df_busi_round7['open']== True]
length = len(df_busi_round7_open)
# then get only restaurant
df_busi_round7_open['is_restaurant'] = 0
df_busi_round7_open['categories'] = df_busi_round7_open['categories'].apply(lambda d: d if isinstance(d, str) else ["None"])
print("fillna finished...")
for i in range(len(df_busi_round7_open)):
    if 'Restaurants' in df_busi_round7_open.categories.iloc[i]:
        df_busi_round7_open.is_restaurant.iloc[i] = 1
    if i % 1000 == 0:
        print(i, "out of", length,"have been already processed...")

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, valu

fillna finished...
0 out of 73210 have been already processed...


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


1000 out of 73210 have been already processed...
2000 out of 73210 have been already processed...
3000 out of 73210 have been already processed...
4000 out of 73210 have been already processed...
5000 out of 73210 have been already processed...
6000 out of 73210 have been already processed...
7000 out of 73210 have been already processed...
8000 out of 73210 have been already processed...
9000 out of 73210 have been already processed...
10000 out of 73210 have been already processed...
11000 out of 73210 have been already processed...
12000 out of 73210 have been already processed...
13000 out of 73210 have been already processed...
14000 out of 73210 have been already processed...
15000 out of 73210 have been already processed...
16000 out of 73210 have been already processed...
17000 out of 73210 have been already processed...
18000 out of 73210 have been already processed...
19000 out of 73210 have been already processed...
20000 out of 73210 have been already processed...
21000 out

In [29]:
df_busi_round7_open_restaurant = df_busi_round7_open[df_busi_round7_open['is_restaurant']==1]
# give open restaurant label 1
df_busi_round7_open_restaurant['open_round7'] = 1
print(len(df_busi_round7_open_restaurant))
# get 20124 open restaurant from round 7


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
df_busi_round7_open_restaurant.drop(['Unnamed: 13', 'is_restaurant'],1, inplace=True)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [44]:
# found that their id is not same...... 
# thought: zip name, city, int(latitude), int(longtitude)
# if all same, then identify as the same restaurant
df_busi_round7_open_restaurant.head()

,type,business_id,name,neighborhoods,full_address,city,state,latitude,longitude,stars,review_count,open,categories,open_round7
2,business,mVHrayjG3uZ_RLHkLj-AMg,Emil's Lounge,NaN,"414 Hawkins Ave Braddock, PA 15104",Braddock,PA,40.408830,-79.866211,4.5,26,True,Bars;American (New);Nightlife;Lounges;Restaurants,1
4,business,KayYbHCt-RkbGcPdGOThNg,Alexion's Bar & Grill,Greentree,"141 Hawthorne St Greentree Carnegie, PA 15106",Carnegie,PA,40.415486,-80.067549,4.0,23,True,Bars;American (Traditional);Nightlife;Restaurants,1
11,business,wJr6kSA5dchdgOdwH6dZ2w,Kings Family Restaurant,NaN,"2100 Washington Pike Carnegie, PA 15106",Carnegie,PA,40.387512,-80.093215,3.5,10,True,Burgers;Breakfast & Brunch;American (Tradition...,1
16,business,fNGIbpazjTRdXgwRY_NIXA,Rocky's Lounge,NaN,"1201 Washington Ave Carnegie, PA 15106",Carnegie,PA,40.396469,-80.084942,4.0,10,True,Bars;American (Traditional);Nightlife;Lounges;...,1
17,business,b9WZJp5L1RZr4F1nxclOoQ,Gab & Eat,NaN,"1073 Washington Ave Carnegie, PA 15106",Carnegie,PA,40.398001,-80.083458,4.5,69,True,Breakfast & Brunch;Sandwiches;Restaurants,1


In [143]:
df_busi_round7_open_restaurant['identify'] = list(zip(df_busi_round7_open_restaurant.name.str.lower(), \
                                                      df_busi_round7_open_restaurant.city.str.lower(), \
                                                      df_busi_round7_open_restaurant.latitude.round(3), \
                                                      df_busi_round7_open_restaurant.longitude.round(3)))
df_busi_round10.latitude.fillna(0, inplace=True)
df_busi_round10.longitude.fillna(0, inplace=True)
df_busi_round10['identify'] = list(zip(df_busi_round10.name.str.lower(), \
                                       df_busi_round10.city.str.lower(), \
                                       df_busi_round10.latitude.round(3), \
                                       df_busi_round10.longitude.round(3)))

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [147]:
df_busi_round10_cut = df_busi_round10[['address', 'is_open','identify']]

In [148]:
temp = df_busi_round7_open_restaurant.merge(df_busi_round10_cut, on="identify")

In [155]:
temp.rename(columns={'is_open': 'open_round10'}, inplace=True)


In [161]:
# temp.drop(['open'], 1, inplace=True)
temp.to_csv("business_round7_paired.txt", sep='\t', encoding='utf-8', index=False)

In [1]:
'''
0408
aim to get corresponding reviews 
'''
import pandas as pd
import numpy as np
business = pd.read_table("business_round7_paired.txt")

In [2]:
review = pd.read_table("/Users/luxiaopeng/notebook/inls613/closure_prediction/round7/review_round7.txt")

In [5]:
len(business[business["open_round10"]==0])

1047

In [11]:
# aim to sample 1047 with open_round10 == 1
business_open_round10 = business[business["open_round10"]==1]
business_close_round10 = business[business["open_round10"]==0]
business_open_round10 = business_open_round10.sample(len(business_close_round10))


In [15]:
business = business_open_round10.append(business_close_round10)

In [19]:
len(business)

2094

In [1]:
business.head()

NameError: name 'business' is not defined

In [22]:
review.head()

,type,review_id,business_id,user_id,stars,text,date,funny,useful,cool
0,review,KPvLNJ21_4wbYNctrOwWdQ,5UmKMjUEUNdYWqANhGckJw,Iu6AxdBYGR4A0wspR9BYHA,5,Excellent food. Superb customer service. I mis...,2014-02-13,0,0,0
1,review,fFSoGV46Yxuwbr3fHNuZig,5UmKMjUEUNdYWqANhGckJw,auESFwWvW42h6alXgFxAXQ,5,Yes this place is a little out dated and not o...,2015-10-31,1,1,0
2,review,pVMIt0a_QsKtuDfWVfSk2A,5UmKMjUEUNdYWqANhGckJw,qiczib2fO_1VBG8IoCGvVg,3,PROS: Italian hoagie was delicious. Friendly ...,2015-12-26,0,0,0
3,review,AEyiQ_Y44isJmNbMTyoMKQ,5UmKMjUEUNdYWqANhGckJw,qEE5EvV-f-s7yHC0Z4ydJQ,2,First the only reason this place could possibl...,2016-04-08,0,1,0
4,review,V-bqYx62zpxfH2oFkzXPzw,5UmKMjUEUNdYWqANhGckJw,jBoH6qKGO7wdYyg_YjBcQA,1,"Normally, I do not do reviews of an establishm...",2016-04-10,0,0,0


In [24]:
busi_id = business[["business_id"]]

In [27]:
review_balanced = review.merge(busi_id, on = "business_id")

In [28]:
len(review_balanced)

126715

In [29]:
len(review)

2684489

In [31]:
review_balanced.tail()

,type,review_id,business_id,user_id,stars,text,date,funny,useful,cool
126710,review,kp-sJBWJlxq96-sxnZpWgA,ECRjdXYLhGzQjCoZKeq6qg,3fcelViwPD5OwMZ9syWhUA,1,We visited this place on on Thursday. I have ...,2016-07-17,1,2,2
126711,review,fgoQEH8y_EwqN0FGFU-WiA,ECRjdXYLhGzQjCoZKeq6qg,BVGlfC3c17JOGl4F3pe-Kw,2,Just thought I try this new AYCE place. I don...,2016-07-18,0,0,0
126712,review,3x9W1P4c21vXWrL9QhEDDw,hNX62vEzvdVsMUMmYYHoPQ,93fNqAY9-y0sADEJV5k2SA,4,"Recently went to Greathouse sports grill, whic...",2016-06-29,0,0,0
126713,review,dFzO7ClyenCLJmcGAKPpXg,hNX62vEzvdVsMUMmYYHoPQ,6ySQiEyjMBIvzpPkwDA9kQ,1,This is the 3rd time I have tried them for bre...,2016-07-12,1,2,0
126714,review,3FGB1o9iObsH3woADkL1RQ,hNX62vEzvdVsMUMmYYHoPQ,uXsvEPbgqcTUjEBthFaKkQ,1,Waiting one whole hour for two breakfast plate...,2016-07-17,0,1,0


In [32]:
business.to_csv("business_final.txt", sep = '\t', encoding= 'utf-8', index = False)
review_balanced.to_csv("review_final.txt", sep = '\t', encoding= 'utf-8', index = False)

In [33]:
review_balanced.sort_values('date')

,type,review_id,business_id,user_id,stars,text,date,funny,useful,cool
18419,review,prvnC11Jwu4-kkqlvjerGw,3nwskbfFgsSjVe6T8keTeg,8ITVDdfK07owxCA1x878Vw,5,Easily my favorite place to eat in Madison. G...,2005-03-03,0,1,0
11374,review,xUEI6GDl2ecFA_KdfqG7hg,XsDFvUcFxnOEOw9zKvO-iA,8ITVDdfK07owxCA1x878Vw,3,"Good ""Mexican"" food. Fast. Cheap. A great p...",2005-03-14,0,1,1
18633,review,CPWOKx-zVsXCEEjtms6JzA,NITVSI02lVfCujdwKw-w4Q,8ITVDdfK07owxCA1x878Vw,4,"Used to be Curve II (yes, there, at one point ...",2005-04-04,0,0,0
18205,review,i_YHEevfnQTB9vS65bc7qw,QnnssZJOLZquRp0hqeK1dA,PwVsKkEYHdp9yIFeMQ6ARA,3,I remember going here a long time ago and not ...,2005-05-25,0,0,0
18295,review,QA5344FEu_-SRpHT-aa5Xw,5xk2-s_QsnHZxycl5yQ4Gg,rRgKbX_VlOnZ_vz4Isxb0Q,5,This place serves both dim sum and a regular m...,2005-06-16,0,0,0
18717,review,P6K6o0wEjDYrR42gRuJcbQ,UO80ofDOenKF4QWR2z-oKA,mA-JeZ5IaTQfKzNAeCYUoQ,5,"Andre's Monte Carlo, voted the #1 restaurant i...",2005-07-12,0,6,3
18671,review,fDhtgxjoAc_eFyy137nYuQ,f0gIqoBYj7GKOh6Icl8vAg,8ITVDdfK07owxCA1x878Vw,5,Five things you should know about Culver's: ...,2005-08-08,2,2,0
18420,review,_R9w1SOK70k8H_6AWFpDNw,3nwskbfFgsSjVe6T8keTeg,6mtHi0gW1AIm4Shdahdd9g,5,The best restaurant in Madison. Get the house...,2005-10-11,0,1,0
11317,review,2BMJrM4-P0ITMjnxX3PGdw,FoHNVQQbiuY7CV8cc-iFzg,m3X7ASscTaIDPnB3h9H71Q,4,"I like the burgers, cheese curds, and darts at...",2005-10-17,0,0,0
18421,review,jIDtMqml4a--lk6ZYwpz3g,3nwskbfFgsSjVe6T8keTeg,m3X7ASscTaIDPnB3h9H71Q,4,Number 24. I wish I'd been the one that order...,2005-10-24,0,1,1


In [34]:
# check the last date of review_round10 and review_round7
round10 = pd.read_table("/Users/luxiaopeng/Documents/aProject/Kim/data/review_round10.txt")
round7 = pd.read_table("/Users/luxiaopeng/notebook/inls613/closure_prediction/round7/review_round7.txt")

In [35]:
round10.sort_values('date')

,cool,business_id,funny,user_id,stars,date,text,review_id,useful
3876463,0,uz7UbvVUwsg68Rok6kbqRg,0,le_brG6cwrzvWdKEGqA7YA,5,2004-07-22,"Sehr gutes Restaurant, leckeres essen und nett...",PbIY2aIyszb6he6J-ey67w,0
1357437,0,9X-43jnj6-6ZBuBdFm7BLA,0,w_6miJytUt6z8oRkGjVG-A,2,2004-09-15,Viel Auswahl täuscht über die wahre Tatsache h...,03B9-gqbeGoMmPJbNzNT5w,0
183111,1,PD2MAlYYi9HCqPH7IBKwTg,0,sE3ge33huDcNJGW3V4obww,5,2004-10-12,Hole in the wall burrito joint with the BEST b...,_CRpX4FGBkxie_1q0-DbjQ,1
752433,0,EZOoB2D8uQHV_gJoGCMTxQ,0,nkN_do3fJ9xekchVC-v68A,4,2004-10-19,Love their subs. Cheap and top shelf ingredients.,1Iobyi_7BkFON25Oegs0aw,0
1709746,0,oYMsq2Xvzw6UbrIlMWjb-A,0,nkN_do3fJ9xekchVC-v68A,4,2004-10-19,Love their pizza. They used to have a great ta...,2F5J51OYtD49eyIUKJKVgg,0
2867577,4,na4Th5DrNauOv-c43QQFvA,2,nkN_do3fJ9xekchVC-v68A,5,2004-10-19,The gold standard for casinos everywhere. Just...,xW294l3Lwh0cxlHU1jwRDA,4
476764,0,AtLv64FV-Pw6JuT3XUKU1g,0,nkN_do3fJ9xekchVC-v68A,4,2004-10-19,Pokey Sticks are the best!,pho1XNCTeRxQVzWR_5vacg,0
1206465,0,GCRvrxMSC1nzShyM4Y-guQ,0,yYSBB5q7bY-qSVvmMgk4FA,5,2004-10-19,"New club in the Hard Rock, and currently the h...",0O7kc-T1SrZpF84jMOyeOg,0
1027989,0,u8C8pRvaHXg3PgDrsUHJHQ,0,c6HT44PKCaXqzN_BdgKPCw,5,2004-10-19,Best pizza I've ever had. My favorite is the ...,JDBubAcRw4FXfg1c5xk-dA,2
1028011,0,u8C8pRvaHXg3PgDrsUHJHQ,1,nkN_do3fJ9xekchVC-v68A,4,2004-10-19,Good stuff. Pricey by normal pizza standards.,0QHCY_55TFHHvyumEMpDew,0


In [36]:
round7.sort_values('date')

,type,review_id,business_id,user_id,stars,text,date,funny,useful,cool
539717,review,nrMMnyveDrtlkNuMygY0SQ,zMN8UGd1zDEreT58OCdnyg,fHtTaujcyKvXglE33Z5yIw,5,The best pizza on the planet. Seriously.,2004-10-12,0,0,0
539995,review,-l40q-y-wMrAQt5S1w8IQg,laMD1HamiWT7pKRXDwjasA,fHtTaujcyKvXglE33Z5yIw,5,Hole in the wall burrito joint with the BEST b...,2004-10-12,0,1,1
569912,review,IVzUfZ8s7ygIrYoZi9WKOg,-7yF42k0CcJhtPw51oaOqQ,rpOyqD_893cqmDAtJLbdog,5,The gold standard for casinos everywhere. Just...,2004-10-18,2,4,3
539718,review,HdODbQJU0D814XdwBxXRfw,zMN8UGd1zDEreT58OCdnyg,18kPq7GPye-YQ3LyKyAZPw,5,Best pizza I've ever had. My favorite is the ...,2004-10-19,0,2,0
540022,review,_Y5m_KLBMs7GHipGa9yxWQ,qUWDPFWaW0fOY66lL0diqg,rpOyqD_893cqmDAtJLbdog,4,Love their pizza. They used to have a great ta...,2004-10-19,0,0,0
539719,review,442pYCJeVzpxovoa7kx-bQ,zMN8UGd1zDEreT58OCdnyg,rpOyqD_893cqmDAtJLbdog,4,Good stuff. Pricey by normal pizza standards.,2004-10-19,1,0,0
540012,review,rMZOMR3SvZJt4DwO2NaUFA,leH24AjTrmcImAbsdh4HFQ,rpOyqD_893cqmDAtJLbdog,4,Love their subs. Cheap and top shelf ingredients.,2004-10-19,0,0,0
540047,review,7726OI-tv7f2MUohXfsbeQ,bibDVZAHz4Q9TLn1iRwlbA,rpOyqD_893cqmDAtJLbdog,4,Pokey Sticks are the best!,2004-10-19,0,0,0
548413,review,gxFkxUPxBGScHZ1NV1QccQ,fl5HJefAH7N3Fp6O2vdGvQ,8VJUJbrfPYJC0vMAtXGR_g,5,top notch but don't plan on leaving with any c...,2004-10-19,0,0,0
1563050,review,17iMU8SmcEGFVJR8LoAq2A,A4FkXoTFi8es0cvrRUMIZg,_PkcFrsTe8Ng-XvzZ4h3kQ,5,"Very fair prices on leather resole ($30), and ...",2004-10-24,0,0,0
